In [57]:
# pip install tensorflow

In [58]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import cv2

import warnings
warnings.filterwarnings("ignore")

from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.utils import load_img
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, concatenate, Conv2DTranspose
import random
import glob

from sklearn.model_selection import train_test_split

In [59]:
print(tf.__version__)

2.11.0


In [60]:
masks = glob.glob("/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/*/*_mask.png")

In [61]:
masks[:4]

['/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/benign/benign (166)_mask.png',
 '/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/benign/benign (290)_mask.png',
 '/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/benign/benign (430)_mask.png',
 '/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/benign/benign (89)_mask.png']

In [62]:
images = [mask_images.replace("_mask", "") for mask_images in masks]

In [63]:
images[:4]

['/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/benign/benign (166).png',
 '/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/benign/benign (290).png',
 '/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/benign/benign (430).png',
 '/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/benign/benign (89).png']

In [64]:
series = list(zip(images, masks))

In [65]:
series[:4]

[('/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/benign/benign (166).png',
  '/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/benign/benign (166)_mask.png'),
 ('/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/benign/benign (290).png',
  '/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/benign/benign (290)_mask.png'),
 ('/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/benign/benign (430).png',
  '/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/benign/benign (430)_mask.png'),
 ('/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/benign/benign (89).png',
  '/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/benign/benign (89)_mask.png')]

In [66]:
dataset = pd.DataFrame(series, columns=['image_path', 'mask_path'])

In [67]:
dataset

,image_path,mask_path
0,/kaggle/input/breast-ultrasound-images-dataset...,/kaggle/input/breast-ultrasound-images-dataset...
1,/kaggle/input/breast-ultrasound-images-dataset...,/kaggle/input/breast-ultrasound-images-dataset...
2,/kaggle/input/breast-ultrasound-images-dataset...,/kaggle/input/breast-ultrasound-images-dataset...
3,/kaggle/input/breast-ultrasound-images-dataset...,/kaggle/input/breast-ultrasound-images-dataset...
4,/kaggle/input/breast-ultrasound-images-dataset...,/kaggle/input/breast-ultrasound-images-dataset...
...,...,...
775,/kaggle/input/breast-ultrasound-images-dataset...,/kaggle/input/breast-ultrasound-images-dataset...
776,/kaggle/input/breast-ultrasound-images-dataset...,/kaggle/input/breast-ultrasound-images-dataset...
777,/kaggle/input/breast-ultrasound-images-dataset...,/kaggle/input/breast-ultrasound-images-dataset...
778,/kaggle/input/breast-ultrasound-images-dataset...,/kaggle/input/breast-ultrasound-images-dataset...


In [68]:
train, test= train_test_split(dataset, test_size=0.25)

In [69]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator()

train_image_gen = ImageDataGenerator(rescale=1./255).flow_from_dataframe(
    dataframe=train, 
    directory=None,
    x_col='image_path',
    y_col=None,
    target_size=(256, 256),
    batch_size=8,
    shuffle=True,
    class_mode=None
)

train_mask_gen = ImageDataGenerator(rescale=1./255).flow_from_dataframe(
    dataframe=train,
    directory=None,
    x_col='mask_path',
    y_col=None,
    target_size=(256, 256), 
    batch_size=8,
    shuffle=True,
    class_mode=None
)

val_image_gen = ImageDataGenerator(rescale=1./255).flow_from_dataframe(
    dataframe=test,
    directory=None, 
    x_col='image_path',
    y_col=None,
    target_size=(256, 256),
    batch_size=8,
    shuffle=False,
    class_mode=None
)

val_mask_gen = ImageDataGenerator(rescale=1./255).flow_from_dataframe(
    dataframe=test,
    directory=None,
    x_col='mask_path',
    y_col=None,
    target_size=(256, 256),
    batch_size=8,
    shuffle=False,
    class_mode=None
)

Found 585 validated image filenames.
Found 585 validated image filenames.
Found 195 validated image filenames.
Found 195 validated image filenames.


In [70]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate
from tensorflow.keras.models import Model

class UNet:
    def __init__(self, input_shape=(256, 256, 3), num_classes=1):
        self.input_shape = input_shape
        self.num_classes = num_classes
        self.model = self.build_model()

    def conv_block(self, inputs, filters, kernel_size=3, activation='relu', padding='same'):
        conv = Conv2D(filters, kernel_size, activation=activation, padding=padding)(inputs)
        conv = Conv2D(filters, kernel_size, activation=activation, padding=padding)(conv)
        return conv

    def build_model(self):
        inputs = Input(self.input_shape)

        # Encoder
        conv1 = self.conv_block(inputs, 64)
        pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

        conv2 = self.conv_block(pool1, 128)
        pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

        conv3 = self.conv_block(pool2, 256)
        pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

        conv4 = self.conv_block(pool3, 512)
        pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

        conv5 = self.conv_block(pool4, 1024)

        # Decoder
        up6 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(conv5)
        concat6 = concatenate([conv4, up6], axis=3)
        conv6 = self.conv_block(concat6, 512)

        up7 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv6)
        concat7 = concatenate([conv3, up7], axis=3)
        conv7 = self.conv_block(concat7, 256)

        up8 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv7)
        concat8 = concatenate([conv2, up8], axis=3)
        conv8 = self.conv_block(concat8, 128)

        up9 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv8)
        concat9 = concatenate([conv1, up9], axis=3)
        conv9 = self.conv_block(concat9, 64)

        # Output
        outputs = Conv2D(self.num_classes, (1, 1), activation='sigmoid')(conv9)

        model = Model(inputs=inputs, outputs=outputs)
        return model


In [73]:
model = UNet()
model.build_model()

model.model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# tf.keras.utils.plot_model(model.model, './model_plot.png', show_shapes = True)

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

train_gen = zip(train_image_gen, train_mask_gen)
val_gen = zip(val_image_gen, val_mask_gen)

num_samples = len(train_image_gen)
steps_per_epoch = num_samples // 8

history = model.model.fit(train_gen, 
                    validation_data=val_gen,
                    epochs=10,
                    steps_per_epoch=steps_per_epoch,      
                    callbacks=[early_stopping], 
                    verbose=1)

Epoch 1/10
9/9 [==============================] - ETA: 0s - loss: 0.6780 - accuracy: 0.7293

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Loss')

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Accuracy')

plt.show()